**load flight and airport**

In [0]:
%python
import datetime

current_date = (datetime.datetime.now()).strftime("%Y-%m-%d")
flight_df=spark.read.format("delta").option("header", "true").option("inferSchema", "true").load(f"/mnt/silverlayer/sampleDatasets/flights2/")
flight_df.display()
airport_df=spark.read.format("delta").option("header", "true").option("inferSchema", "true").load(f"/mnt/silverlayer/sampleDatasets/airport/{current_date}/")


**transform data**

In [0]:
import pyspark.sql.functions as F

# Create a new Date column
flight_df=flight_df.withColumn("Date", F.to_date(
    F.concat(
        F.col("Year").cast("string"), 
        F.lpad(F.col("Month").cast("string"), 2, "0"),  # Padding for Month
        F.lpad(F.col("DayofMonth").cast("string"), 2, "0")  # Padding for DayofMonth
    ), "yyyyMMdd")
)

In [0]:

flight_df=flight_df.drop("Year", "Month", "DayofMonth","DayOfWeek")
flight_df2=flight_df.select("Date","arrdelay","depdelay","origin","Cancelled","CancellationCode","UniqueCarrier","FlightNum","TailNum","DepTime")
flight_df2.display()


**create database and table**

In [0]:
%sql
create database if not exists `goldlayer-myproject`.goldlayerDB

In [0]:
%sql
use `goldlayer-myproject`.goldlayerDB;
describe database goldlayerDB

In [0]:
%sql
CREATE TABLE IF NOT EXISTS goldlayerDB.DIM_Flight (
    Date date,
    arrdelay INTEGER,
    depdelay INTEGER,
    origin STRING,
    Cancelled INTEGER,
    CancellationCode STRING,
    UniqueCarrier STRING,
    FlightNum INTEGER,
    TailNum STRING,
    DepTime STRING
) 
USING DELTA
PARTITIONED BY (YEAR(Date));


**create temp view for dataframe** 

In [0]:
%python
flight_df2.createOrReplaceTempView("flight_temp_view")

**avoid duplicate data and load to azure storage**

In [0]:
%sql
MERGE INTO goldlayerDB.DIM_Flight AS target
USING flight_temp_view AS source
ON target.Date = source.Date
   AND target.FlightNum = source.FlightNum
   AND target.UniqueCarrier = source.UniqueCarrier
WHEN NOT MATCHED THEN
   INSERT (Date, arrdelay, depdelay, origin, Cancelled, CancellationCode, UniqueCarrier, FlightNum, TailNum, DepTime)
   VALUES (source.Date, TRY_CAST(source.arrdelay AS INT), TRY_CAST(source.depdelay AS INT), source.origin,
           TRY_CAST(source.Cancelled AS INT), source.CancellationCode, source.UniqueCarrier, source.FlightNum,
           source.TailNum, source.DepTime);


In [0]:

# Read data from Unity Catalog table and write to ADLS Gen 2
spark.read.table("goldlayerDB.DIM_Flight").write.format("delta").mode("overwrite").save(f"/mnt/goldlayer/sampleDatasets/flights2/")

### airport data

In [0]:
%sql
CREATE TABLE IF NOT EXISTS goldlayerDB.DIM_Airport (
    code STRING,
    description  STRING,
    city STRING,
    state STRING,
    airport_name STRING
) 
USING DELTA
PARTITIONED BY (City);


In [0]:
airport_df.createOrReplaceTempView("airport_temp_view")

In [0]:
%sql
merge into goldlayerDB.DIM_Airport as target
using airport_temp_view as source
on target.code = source.Code and
    target.airport_name = source.name and 
    target.city = source.City
when not matched then insert (code, description, city, state, airport_name) values (source.Code, source.Description, source.City, source.State, source.name);


In [0]:
# Read data from Unity Catalog table and write to ADLS Gen 2
spark.read.table("goldlayerDB.DIM_Airport").write.format("delta").mode("overwrite").save(f"/mnt/goldlayer/sampleDatasets/airport/")

In [0]:
%sql
SHOW EXTERNAL LOCATIONS;




In [0]:
%sql
-- select * from `goldlayer-myproject`.goldlayerDB.dim_airport limit 10;
show tables in `goldlayerDB`.`dim_airport`;
